In [77]:
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import f1_score

In [58]:
df = pd.read_csv('data/working_clean.csv')
df.dropna(inplace=True)
df.drop(columns='track_id', inplace=True)

In [59]:
df.head()

,acousticness,danceability,duration_ms,energy,explicit,genre,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,year
0,0.0999,0.507,249787.0,0.446,0,1,0.000025,7,0.1060,-7.337,1,0.0587,141.647,2,0.285,2015
1,0.2050,0.722,203413.0,0.681,0,1,0.000000,10,0.0501,-4.419,1,0.0330,138.008,2,0.965,2017
2,0.4700,0.602,259133.0,0.661,0,1,0.003930,8,0.1460,-8.615,1,0.0631,117.884,2,0.341,2016
3,0.0987,0.833,234867.0,0.309,0,1,0.000009,7,0.0819,-10.400,1,0.1050,87.047,2,0.927,2009
4,0.1140,0.621,214297.0,0.664,0,1,0.000000,9,0.1240,-5.609,0,0.0734,132.220,2,0.480,2003


In [45]:
# df['genre'].value_counts()

In [46]:
# df2 = df.drop(columns=['genre'], axis=1)
# df2 = pd.get_dummies(data=df)
# df2['genre'] = df['genre']
# df2.dropna(inplace=True)

In [60]:
y = df['genre']
X = df.drop(columns=['genre'], axis=1)

In [48]:
# lb = preprocessing.LabelBinarizer()
# lb.fit(y)

In [49]:
# lb.transform(y)

In [50]:
# lb.classes_

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=123)

In [52]:
# df = pd.DataFrame(y_test)

In [53]:
# df

In [56]:
df[0]

KeyError: 0

In [113]:
# Construct some pipelines
pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('hot', OneHotEncoder(categories='auto', handle_unknown='ignore')),
                    ('clf', OneVsRestClassifier(LogisticRegression(random_state=42)))])

pipe_dt = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', tree.DecisionTreeClassifier(random_state=42))])

pipe_rf = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('clf', RandomForestClassifier(random_state=42))])

# List of pipelines for ease of iteration
pipelines = [pipe_lr, pipe_dt, pipe_rf]

# Dictionary of pipelines and classifier types for ease of reference
pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Random Forrest'}

# Fit the pipelines
for pipe in pipelines:
    pipe.fit(X_train, y_train)
#     pipe.predict(X_test)

# # Compare accuracies
for idx, val in enumerate(pipelines):
#     print(val.f1_score())
     print('%s pipeline test accuracy: %.3f' % (pipe_dict[idx], val.score(X_test, y_test)))

# # Identify the most accurate model on test data
best_acc = 0.0
best_clf = 0
best_pipe = ''
for idx, val in enumerate(pipelines):
    if val.score(X_test, y_test) > best_acc:
        best_acc = val.score(X_test, y_test)
        best_pipe = val
        best_clf = idx
        
# # Identify the most accurate model on test data
# best_acc = 0.0
# best_clf = 0
# best_pipe = ''
# for idx, val in enumerate(pipelines):
#     if val.score(X_test, y_test) > best_acc:
#         best_acc = val.score(X_test, y_test)
#         best_pipe = val
#         best_clf = idx        

    
# print('Classifier with best accuracy: %s' % pipe_dict[best_clf])

# # Save pipeline to file
# joblib.dump(best_pipe, 'best_pipeline.pkl', compress=1)
# print('Saved %s pipeline to file' % pipe_dict[best_clf])



//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Logistic Regression pipeline test accuracy: 0.363
Decision Tree pipeline test accuracy: 0.705
Random Forrest pipeline test accuracy: 0.765


In [115]:

pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('pca', PCA(n_components=2)),
                    ('hot', OneHotEncoder()),
                    ('clf', OneVsRestClassifier(LogisticRegression(random_state=42)))])


# # List of pipelines for ease of iteration
# pipelines = [pipe_dt, pipe_rf]

# # Dictionary of pipelines and classifier types for ease of reference
# pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Random Forrest'}

pipe_lr.fit(X_train, y_train)
#     pipe.predict(X_test)

# # # Compare accuracies
# for idx, val in enumerate(pipelines):
# #     print(val.f1_score())
     
# print('%s pipeline test accuracy: %.3f' % (pipe_dict[idx], val.score(X_test, y_test)))


pipe_lr.score(X_test,y_test)

//anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


ValueError: OneHotEncoder in legacy mode cannot handle categories encoded as negative integers. Please set categories='auto' explicitly to be able to use arbitrary integer values as category identifiers.

In [69]:
pipe_dict

{0: 'Logistic Regression', 1: 'Decision Tree', 2: 'Random Forrest'}